In [1]:
import locale
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd

C:\Users\Bouayad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/paris_02_11_23.csv')

In [5]:
import sweetviz as sv

report = sv.analyze(df)
report.show_html('report.html')


Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [7]:
#afficher les values uniques de la colonne 'language' et la fréquence (en pourcentage) de chaque valeur
df['language'].value_counts(normalize=True)

language
en    1.0
Name: proportion, dtype: float64

In [9]:
#Lister les colonnes du dataframe
df.columns

Index(['review_id', 'date', 'review_rating', 'title', 'text', 'votes', 'url',
       'language', 'platform', 'author_id', 'author_name', 'author_username',
       'name', 'id', 'description', 'rating', 'rating_count', 'features'],
      dtype='object')

In [11]:
#les valeurs de la colonne plateform
df['platform'].value_counts(normalize=True)

platform
OTHER     0.669060
MOBILE    0.330592
TABLET    0.000349
Name: proportion, dtype: float64

In [13]:
#afficher les valeurs de la colonne features des 10 premières hotels (sur la base de la colonne 'id')
df['features'].head(10)

In [15]:
#pour chaque colonne affiche son type 
df.dtypes

review_id            int64
date                object
review_rating        int64
title               object
text                object
votes                int64
url                 object
language            object
platform            object
author_id           object
author_name         object
author_username     object
name                object
id                   int64
description         object
rating             float64
rating_count       float64
features            object
dtype: object

In [16]:
#convertir la colonne 'date' en type datetime
df['date'] = pd.to_datetime(df['date'])


In [17]:
#convertir la colonne rating en type float
df['rating'] = df['rating'].astype(float)

In [19]:
#créer une liste avec les colonnes à supprimer (url, language, platform, author_id, author_name, author_username) et créer une fonction qui supprime les colonne mentionnées dans la liste
to_drop = ['url', 'language', 'platform', 'author_id', 'author_name', 'author_username']
def drop_cols(df, cols):
    df.drop(columns=cols, inplace=True)
drop_cols(df, to_drop)

In [23]:
#nombre de doublons sur la base de la colonne 'review_id'
df.duplicated(subset=['review_id']).sum()





16704

In [26]:
#supprimer les doublons sur la base de la colonne 'review_id'
df.drop_duplicates(subset=['review_id'], inplace=True)